In [1]:
import sys
import os
if not any(path.endswith('textbook') for path in sys.path):
    sys.path.append(os.path.abspath('../../..'))
from textbook_utils import *

In [2]:
co2 = pd.read_csv('data/co2_mm_mlo.txt', header=None, skiprows=72, 
                  sep='\s+',
                  names=['Yr', 'Mo', 'DecDate', 'Avg', 'Int', 'Trend', 'days'])
co2_NA = co2.replace(-99.99, np.NaN)

(ch:wrangling_structure)=
# Modifying Structure

If a dataframe has an inconvenient structure, it can be difficult to do the
analysis that we want. The wrangling process often reshapes the data frame in
some way to make the analysis easier and more natural. These changes can simply
take a subset of the rows and/or columns from the table, change the table's
granularity to reshape in a more fundamental way. In this section, we use the techniques from
{numref}`Chapter %s <ch:pandas>` to show how to modify the structure.

**Simplify Structure.** If a data frame has features that are not needed
in our analysis then we may want to drop these extraneous columns to make
handling the data frame easier. Or, if we want to focus on a particular period
of time or geographic area we may want to take a subset of the rows
[^subsetting]. In {numref}`Chapter %s <ch:files>`, we read into
our data frame a small set of features from the hundreds available in the DAWN
survey because we were interested in understanding the patterns of type of ER
visit by demographics of the patient. Later, in
{numref}`Chapter %s <ch:eda>`, we restrict an analysis of
home sale prices to one year and
a few cities in an effort to reduce the impact of inflation and to better study
the effect of location on sale price. 

[^subsetting]: Subsetting is covered in {numref}`Section %s <ch:pandas_subsetting>` 

**Adjust Granularity.** In an earlier example in this chapter, 
CO2 measurements were aggregated from monthly averages to yearly averages in order to 
better visualize annual trends. In the next section, we provide another example
where we aggregate violation-level data to the inspection-level so that it can
be combined with the restaurant inspection scores. In both of these examples, we adjust
the granularity of the data frame to work with a coarser granularity
by grouping together records and then aggregating values. 

With the CO2 measurements, we grouped the monthly values from the same year, and then
averaged them. Other common aggregations of a group are the number records,
sum, minimum, maximum, and first or last value in the group. The details
of adjusting granularity of pandas data frames can be found in
{numref}`Chapter %s <ch:pandas>`, including how to group by multiple column values.

At times a dataset might have mixed granularity, where records are at different
levels of detail. A common case is in data provided by government agencies
where data at the county and state levels are included in the same file. When
this happens, we usually want to split the data frame into two, one at the
county level and the other at the state level. This makes a county level and
state level analyses much easier, even feasible, to perform.

**Reshape.** Data, especially from government sources, can be shared as pivot
tables. These *wide* tables have data values as column names, and are often
difficult to use in analysis. We may need to reshape them into a *long*
form. {numref}`Figure %s <wide-vs-long>` depicts the same data stored in both
wide and long data tables.

```{figure} figures/wide-vs-long.svg
---
name: wide-vs-long
---

An example of a wide data table (top) and a long data table (bottom). Both
tables contain the same data. Each row of the wide data table maps to three
rows in the long data table, as highlighted in the tables.
Notice that in the wide data table, each row
has three observations, one for each month. In the long data table, each
row only has one observation. Long data tables are generally easier to
aggregate for future analysis.
```

:::{note}

Long form data is also frequently called *tidy data* {cite}`wickhamTidy2014`.

:::

To demonstrate, we can reshape the CO2 data into a wide data frame that is like a pivot table in shape. There is a column for each month and a row for each year.

In [4]:
co2_pivot = pd.pivot_table(
    co2[10:34],
    index='Yr',   # Column to turn into new index
    columns='Mo',  # Column to turn into new columns
    values='Avg', # Column to aggregate for values
    aggfunc=sum)    # Aggregation function

co2_wide = co2_pivot.reset_index()

display_df(co2_wide, cols=8)

Mo,Yr,1,2,3,...,9,10,11,12
0,1959,315.62,316.38,316.71,...,313.84,313.26,314.8,315.58
1,1960,316.43,316.97,317.58,...,314.16,313.83,315.0,316.19


The column headings are months, and the cell values in the grid are the CO2 monthly averages. We can turn this data frame back into a long, aka *tall*, data frame, where the column names become a feature, called month, and the values in the grid are reorganized into a second feature, called average.  

In [14]:
co2_long = co2_wide.melt(id_vars=['Yr'],
                         var_name='month',
                         value_name='average')

display_df(co2_long, rows=4)

,Yr,month,average
0,1959,1,315.62
1,1960,1,316.43
...,...,...,...
22,1959,12,315.58
23,1960,12,316.19


Notice that the data has been recaptured in its original shape (although the
rows are not in their original order). Wide-form data is more common when we
expect readers to look at the data table itself, like in an economics article
or news story. But long-form data is more useful for data analysis. For
instance, `co2_long` lets us write short `pandas` code to group by either year
or month, while the wide-form data makes it difficult to group by year.
The `.melt()` method is particularly useful for converting wide-form into long-form data. 

These structural modifications have focused on a single table. However, we often want to combine information that is spread across multiple tables. In the next section, we combine the techniques introduced in this chapter to wrangle the restaurant inspection data and address joining tables.